# FPGA Visuals

In [ ]:
# Imports
%matplotlib notebook

import os
import random
from collections import defaultdict
import sys
import numpy as np
from scipy import spatial
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import seaborn as sns
import matplotlib.animation as animation

sns.set_style('darkgrid')
np.random.seed(42)

get_colors = lambda length: [color.reshape(1, -1) for color in plt.get_cmap('Spectral')(np.linspace(0, 1.0, length))]

In [ ]:
y_pred_folders = {
    'sga': '/Users/jeffchenchengyi/Downloads/sinx_2019-Nov-17-13:54:06',
    'ns-es': '/Users/jeffchenchengyi/Downloads/sinx_2019-Nov-17-14:00:06',
    'nsr-es': '/Users/jeffchenchengyi/Downloads/sinx_2019-Nov-17-14:00:45'
}
X_test = np.load('/Users/jeffchenchengyi/Downloads/X_test.npy')
y_preds = {strategy: defaultdict() for strategy in y_pred_folders.keys()}
for strategy, folder in y_pred_folders.items():
    for pred in os.listdir(folder):
        y_preds[strategy][int(pred.split('_')[0][3:])] = \
            np.load(os.path.join(folder, pred)).flatten()

In [ ]:
sorted_preds = {strategy: None for strategy in y_preds.keys()}

for strategy, predictions in y_preds.items():
    sorted_preds[strategy] = sorted(list(zip(predictions.items(), get_colors(len(predictions)))), key=lambda x: x[0])

In [ ]:
fig, ax = plt.subplots(len(sorted_preds.keys()), 1)

for ax_idx, (strategy, predictions) in enumerate(sorted_preds.items()):
    ax[ax_idx].scatter(X_test, np.sin(X_test), c='black', label='ground_truth')
    ax[ax_idx].set_title(strategy)
    for (idx, pred), color in sorted_preds[strategy][::10]:
        ax[ax_idx].scatter(x=X_test, y=pred, c=color, alpha=0.2, label='gen-'+str(idx))

plt.tight_layout()
plt.legend()
plt.show();

## Animate

In [ ]:
x = X_test

In [ ]:
fig, ax = plt.subplots(1, 1)

strategy = 'sga'
ax.scatter(x, np.sin(x), c='black', label='ground_truth')
ind, = ax.plot(x, sorted_preds[strategy][0][0][1], 
               c='g', 
               alpha=0.5, 
               marker='o', 
               linestyle='', 
               label='gen-'+str(0))

def animate(gen_idx):
    ind.set_ydata(sorted_preds[strategy][gen_idx][0][1])  # update the data
    ind.set_label('gen-'+str(gen_idx))
    plt.legend()
    return ind,


# Init only required for blitting to give a clean slate.
def init():
    ind.set_ydata(np.ma.array(x, mask=True))
    return ind,

ani = animation.FuncAnimation(fig=fig, 
                              func=animate, 
                              frames=np.arange(0, 100), init_func=init,
                              interval=100, blit=True, repeat=True)

# Set up formatting for the movie files
plt.title(strategy)
ani.save('sinx_evolve_{}.gif'.format(strategy), writer=animation.PillowWriter(fps=60), dpi=500)

plt.show();

In [ ]:
fig, ax = plt.subplots(1, 1)

strategy = 'ns-es'
ax.scatter(x, np.sin(x), c='black', label='ground_truth')
ind, = ax.plot(x, sorted_preds[strategy][0][0][1], 
               c='g', 
               alpha=0.5, 
               marker='o', 
               linestyle='', 
               label='gen-'+str(0))

def animate(gen_idx):
    ind.set_ydata(sorted_preds[strategy][gen_idx][0][1])  # update the data
    ind.set_label('gen-'+str(gen_idx))
    plt.legend()
    return ind,


# Init only required for blitting to give a clean slate.
def init():
    ind.set_ydata(np.ma.array(x, mask=True))
    return ind,

ani = animation.FuncAnimation(fig=fig, 
                              func=animate, 
                              frames=np.arange(0, 100), init_func=init,
                              interval=100, blit=True, repeat=True)

# Set up formatting for the movie files
plt.title(strategy)
ani.save('sinx_evolve_{}.gif'.format(strategy), writer=animation.PillowWriter(fps=60), dpi=500)

plt.show();

In [ ]:
fig, ax = plt.subplots(1, 1)

strategy = 'nsr-es'
ax.scatter(x, np.sin(x), c='black', label='ground_truth')
ind, = ax.plot(x, sorted_preds[strategy][0][0][1], 
               c='g', 
               alpha=0.5, 
               marker='o', 
               linestyle='', 
               label='gen-'+str(0))

def animate(gen_idx):
    ind.set_ydata(sorted_preds[strategy][gen_idx][0][1])  # update the data
    ind.set_label('gen-'+str(gen_idx))
    plt.legend()
    return ind,


# Init only required for blitting to give a clean slate.
def init():
    ind.set_ydata(np.ma.array(x, mask=True))
    return ind,

ani = animation.FuncAnimation(fig=fig, 
                              func=animate, 
                              frames=np.arange(0, 100), init_func=init,
                              interval=100, blit=True, repeat=True)

# Set up formatting for the movie files
plt.title(strategy)
ani.save('sinx_evolve_{}.gif'.format(strategy), writer=animation.PillowWriter(fps=60), dpi=500)

plt.show();

## Get halloffame weights

In [ ]:
# from deap import base, creator
# import pickle

# with open('/Users/jeffchenchengyi/Downloads/checkpoint_gen99.pkl', "rb") as cp_file:
#     # Define objective, individuals, population, and evaluation
#     creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#     creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)
#     cp = pickle.load(cp_file)

In [ ]:
# cp['halloffame']

In [ ]:
# from keras.datasets import mnist

In [ ]:
# (X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# int(len(X_train)*0.2)

In [ ]:
# np.dot(
#     np.array([
#         [400000, -100000, -200000],
#         [250000, -50000, -75000],
#         [50000, 0, -30000]
#     ]), 
#     np.array([0.25, 0.35, 0.4])
# )